# Overview
Today we'll go over many of the concepts that power the Solvestack's backend code.  Since it uses the Django REST framework, we'll illustrate examples from tutorials from their site along with our Solvestack code.

## Our goals
1. Review terms
1. Review serializers
1. Examine class-based views using Django REST
1. Examine hyperlinked APIs and other relationship styles
1. Show different ways pagination can be used
1. Examine view sets and create new ones

# General terms review
__Pull up the solvestack admin page.  As we work through these, type the answers from the mentees into the glossary terms.__

__CLI__

Stands for command line interface - programs that are run from your terminal

Examples: brew, pip3, manage.py, ls, pwd

__IDE__

A place to view and edit your code

Examples: Visual Studio Code, Atom, JetBrains (various language specific IDEs)

__Library__

A collection of modules

__Module__

methods and classes for a purpose that can be used by other code elsewhere

Examples: shortuuid

__Package__

A neat wrapper for installation and easy calling of a library from other code elsewhere
Package Manager
A tool that easily installs packages for a project OR for your local machine

Examples: pip, npm, npx, homebrew

__Frontend/GUI/UI__

The presentation layer of a given application - what the user sees. Or, GUI, stands for graphical user interface

__Backend__

The business logic layer that interacts with the database layer and serves the data to external clients

__Request__

Client sends this message to the web server to accomplish tasks.

__Response__

Web server gives this message to let the client know the result and the status of their task.

__API__

Stands for application programming interface. Fundamental way of interaction between two systems or two libraries, and more.

__Framework__

A collection of packages and tooling that make life easier when coding for a specific purpose

Examples: React, Vue, Django, Django REST Framework

__Database__

Place to store all records and write records to in an organized and optimized way

Examples: SQLite, MySQL, PostgreSQL, MongoDB

__Web Server__

The program used to broadcast our code on a given port and address.

Example: localhost is the address of our machine, and 3000 is the default address for React and other frontends

__Client__

A program or API used to receive the data broadcasted by the web server

Examples: a browser like Chrome/Mozilla/Safari, or Postman client, or a client in a given language like Python or Axios is a commonly used client in JS

__Physical Server__

The physical machine that runs the web server and can house a database and your files that contain your code.

Examples: your laptop, a server in a datacenter

__Frontend Component__

A template to generate HTML that’s reusable

__SQL__

SQL is a standard language for storing, manipulating and retrieving data in relational databases.

__IDE__

Integrated Development Environment - a fancy name for a fancy code editor

Examples: VSCode

__Virtual Environment__

An isolated location so that each project can have its own dependencies, regardless of what dependencies every other project has.

__Protocol__

Standard structure of network requests and responses on the web that the whole world has agreed upon

Examples usually end in “protocol”:
HTTP, HTTPS, RDP (Remote Desktop Protocol to get into a Windows Server), NTP (Network Time Protocol)

__Contract__

Specifications the developer makes for an API to adhere to in order to provide consistency for downstream stakeholders of the API.


__Future__

DNS
Hostsfile


# Django REST terms review

__Url resources endpoint__

Defines the viewset used and the endpoint used

__Viewset__

Defines our filters for the resources and all the HTTP actions we want to use (“GET”, “POST”, “PUT”, “PATCH”, and “DELETE”

Gives response codes and JSON from the serializer into the response body.

__Serializer__

Example for POST action:
Deserializes our JSON input from our request body into the ORM equivalent for writing actions - and response serializes the newly created resource from our ORM back into JSON response.

__ORM__

Object Representation Model - Django’s way of interfacing (API) between the database and our Python code.

Is used by both Django framework for server side rendering web development AND Django REST Framework libs

__Manager / Queryset__

Manages the objects of a model type and has handy ways to query the database or manipulate querysets and save them to the database. The manager is the standard queryset you start from before adding filters and exclusions, etc.

Django starts you out with its standard manager but we can extend it to prefetch_related or select_related fields. 

__Model__

Blueprint for Django objects that are able to be saved and loaded from the database.

Generates migrations to create tables and define their columns for storage.

__Migrations__

Easy blueprints for manipulating the structure of the database and its tables


__Primary Key__

The main way to identify a given record or model


# Serializer review
We use serialization to convert our data to JSON.
- Show `snippets/serializers.py` code
- Explain how this is similar to Django's forms
- Show serialization in solvestack at `glossary/serializers.py`

## Goals
- look at serializers class of rest_framework
    - field types, serializers.Serializer
- understand imports of snippets.models (`Snippet`, `LANGUAGE_CHOICES`, `STYLE_CHOICES`)

## Activities
- Run some shell commands on tutorial db
- Run some shell commands on solvestack db

### Tutorial serializer demo
- Run tutorial server with `python manage.py runserver`
- Show the admin site [here](http://127.0.0.1:8000/admin/), view the snippets we have loaded
- Open Postman to make API requests and show serialization of data we just looked at

In [ ]:
# solvestack serializer
from rest_framework import serializers

from solvestack.glossary.models import Term


class TermSerializer(serializers.ModelSerializer):
    class Meta:
        model = Term
        fields = ["id", "name", "definition"]
        read_only_fields = ["id"]

### Solvestack serializer demo
- Run solvestack server with `python manage.py runserver`
- Show the admin site [here](http://127.0.0.1:8000/admin/), view the terms we have loaded
- Open Postman to make API requests and show serialization of data we just looked at: run a GET on `http://127.0.0.1:8000/terms/`

Output from initial JSON fixture import:
```
[
    {
        "id": "6LWr7wMsmk6rdgvoA9W3dG",
        "name": "frontend",
        "definition": "presentation layer"
    },
    {
        "id": "8JdHcDARgHG9H6CHJEYCwD",
        "name": "backend",
        "definition": "business logic and database access layer"
    }
]
```

# Class-based views
## Goals
- Understand what features mixin classes provide
- Understand how classes can be refactored with already mixed-in generic class-based views

## Activities
- 
Mixins added classes in `snippets/views.py`:

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework import mixins
from rest_framework import generics

class SnippetList(mixins.ListModelMixin,
                  mixins.CreateModelMixin,
                  generics.GenericAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

    def get(self, request, *args, **kwargs):
        return self.list(request, *args, **kwargs)

    def post(self, request, *args, **kwargs):
        return self.create(request, *args, **kwargs)

Using already mixed-in generic class-based view from `snippets/views.py`:

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework import generics


class SnippetList(generics.ListCreateAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

class SnippetDetail(generics.RetrieveUpdateDestroyAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

Solvestack code from `glossary/views.py` uses a viewset, which we'll talk about later.

# More about serialization and relationships
Django REST supports many different types of relationships between data.  Here are some examples:
- Using primary keys
- Using hyperlinking between entities
- Using a unique identifying slug field on the related entity
- Using the default string representation of the related entity
- Nesting the related entity inside the parent representation
- Some other custom representation

## Goals
- Understand differences between `ModelSerializer` and `HyperLinkedModelSerializer`
    - Tie this to previous lessons on serializers (review videos)
- Show other relationship styles

## Activities
- Implement a different relationship style

In [ ]:
# tutorial serializer w/o models
from rest_framework import serializers
from snippets.models import Snippet, LANGUAGE_CHOICES, STYLE_CHOICES


class SnippetSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    title = serializers.CharField(required=False, allow_blank=True, max_length=100)
    code = serializers.CharField(style={'base_template': 'textarea.html'})
    linenos = serializers.BooleanField(required=False)
    language = serializers.ChoiceField(choices=LANGUAGE_CHOICES, default='python')
    style = serializers.ChoiceField(choices=STYLE_CHOICES, default='friendly')

    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return Snippet.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet` instance, given the validated data.
        """
        instance.title = validated_data.get('title', instance.title)
        instance.code = validated_data.get('code', instance.code)
        instance.linenos = validated_data.get('linenos', instance.linenos)
        instance.language = validated_data.get('language', instance.language)
        instance.style = validated_data.get('style', instance.style)
        instance.save()
        return instance

`snippets/serializers.py` with `ModelSerializer`:

In [ ]:
class SnippetSerializer(serializers.ModelSerializer):
    owner = serializers.ReadOnlyField(source='owner.username')
    class Meta:
        model = Snippet
        fields = ['id', 'title', 'code', 'linenos', 'language', 'style', 'owner',]

from `snippets/serializers.py` with `HyperLinkedModelSeralizer`:

In [ ]:
class SnippetSerializer(serializers.HyperlinkedModelSerializer):
    owner = serializers.ReadOnlyField(source='owner.username')
    highlight = serializers.HyperlinkedIdentityField(view_name='snippet-highlight', format='html')

    class Meta:
        model = Snippet
        fields = ['url', 'id', 'highlight', 'owner',
                  'title', 'code', 'linenos', 'language', 'style']


class UserSerializer(serializers.HyperlinkedModelSerializer):
    snippets = serializers.HyperlinkedRelatedField(many=True, view_name='snippet-detail', read_only=True)

    class Meta:
        model = User
        fields = ['url', 'id', 'username', 'snippets']

`ModelSerializer` classes are a shortcut for creating serializer classes:
- automatically determined set of fields
- simple `create()` and `update()` methods are implemented for you

Solvestack's `glossary/serializers.py` with `ModelSerializer`:

In [ ]:
from rest_framework import serializers

from solvestack.glossary.models import Term


class TermSerializer(serializers.ModelSerializer):
    class Meta:
        model = Term
        fields = ["id", "name", "definition"]
        read_only_fields = ["id"]

# Pagination
## Goals
- Understand the REST_FRAMEWORK dictionary in `tutorial/settings.py`
- Understand how PageNumberPagination class works

## Activities
- Look at other pagination classes and show how they work

Tutorial's `settings.py`:

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 10
}

Solvestack's `solvestack/settings.py`:

In [ ]:
REST_FRAMEWORK = {
    "DEFAULT_FILTER_BACKENDS": ("django_filters.rest_framework.DjangoFilterBackend",)
}

# Viewset Exercises
## Goals
- import statements (`viewsets`, `action`, `Response`)
- understand what actions `ReadOnlyModelViewSet` provides
- understand what actions `ModelViewSet` provides
- show other viewset classes
- show code needed for manual urls
- understand how routers work to handle URLConf with router with DefaultRouter().urls
- show other router classes

## Activities
- create a new viewset with this information that adds to the tutorial code
From `snippets/views.py`:

In [ ]:
from rest_framework import viewsets
from rest_framework.decorators import action
from rest_framework.response import Response

class UserViewSet(viewsets.ReadOnlyModelViewSet):
    """
    This viewset automatically provides `list` and `detail` actions.
    """
    queryset = User.objects.all()
    serializer_class = UserSerializer

class SnippetViewSet(viewsets.ModelViewSet):
    """
    This viewset automatically provides `list`, `create`, `retrieve`,
    `update` and `destroy` actions.

    Additionally we also provide an extra `highlight` action.
    """
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer
    permission_classes = [permissions.IsAuthenticatedOrReadOnly,
                          IsOwnerOrReadOnly]

    @action(detail=True, renderer_classes=[renderers.StaticHTMLRenderer])
    def highlight(self, request, *args, **kwargs):
        snippet = self.get_object()
        return Response(snippet.highlighted)

    def perform_create(self, serializer):
        serializer.save(owner=self.request.user)

from `snippets/urls.py`:

In [ ]:
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from snippets import views

# Create a router and register our viewsets with it.
router = DefaultRouter()
router.register(r'snippets', views.SnippetViewSet)
router.register(r'users', views.UserViewSet)

# The API URLs are now determined automatically by the router.
urlpatterns = [
    path('', include(router.urls)),
]

Solvestack's `glossary/views.py`:

In [ ]:
from rest_framework import viewsets

from .serializers import TermSerializer
from .models import Term
from .filters import TermsFilter


class TermViewSet(viewsets.ModelViewSet):
    queryset = Term.objects.all()
    serializer_class = TermSerializer
    filter_class = TermsFilter

Solvestack's `glossary/urls.py`:

In [ ]:
from rest_framework import routers

from django.contrib import admin
from django.urls import path, include

from solvestack.glossary import views

router = routers.DefaultRouter()
router.register("terms", views.TermViewSet)

urlpatterns = [
    path("", include(router.urls)),
    path("admin/", admin.site.urls),
]